#🧩**Звіт проведення Етапу 2: "Попередня Обробка Даних" (обробка пропусків, кодування, масштабування)**

**Project:** Прогнозування Відтоку Клієнтів  
**Author:** Natalia Kalashnikova  
**Team Lead:** Vitalii Subbotin  
**Date:** 2025-10-14
---

## 🎯 **Мета етапу**

---

**Мета проєкту:** прогноз відтоку клієнтів (**churn**) на основі історичних даних телекомунікаційної компанії.  
**Поточний етап:** Попередня Обробка Даних (**Data Preprocessing & Feature Engineering**) — обробка пропусків, кодування, масштабування.


## 🔍 **Опис етапів проведення обробки даних**
---

### **1.1. Базовий Аналіз та Перевірки**
* **Завантаження та огляд даних:** Проаналізовано типи даних, виявлено відсутність категоріальних ознак.
* **Перевірка дублікатів:** Встановлено відсутність повних дублікатів.
* **Аналіз пропущених значень:** Виявлено пропущені значення у трьох ознаках: `reamining_contract` (майже 30%), `download_avg` та `upload_avg` (менше 1%).

### **1.2. Обробка Відсутніх Значень та Формування Датасетів**
Сформовано кілька тестових датасетів для порівняння різних стратегій обробки пропусків та ознак:
1.  **Стратегія Видалення:**
    * Видалено записи з пропусками в `download_avg` та `upload_avg` (через малу кількість пропусків).
    * **`df2`:** Додатково видалено записи з пропусками в `reamining_contract`.
2.  **Стратегія Заповнення та Додавання Ознаки (Feature Engineering):**
    * Додано бінарну ознаку `reamining_contract_missing` (сильна кореляція з `churn`).
    * **`df1_mean`, `df1_median`, `df1_mode`:** Пропуски в `reamining_contract` заповнено відповідно середнім, медіаною та модою.
3.  **Стратегія Видалення Слабокорельованих Ознак:**
    * Сформовано датасети **`df1_mean_drop`** та **`df2_drop`** шляхом видалення ознак `bill_avg` та `service_failure_count` (через слабку кореляцію з цільовою змінною).

### **1.3. Масштабування та Базове Тестування Моделей**
* **Масштабування:** Застосовано **StandardScaler** для стандартизації числових ознак.
* **Тестування Моделей:** Протестовано 6 класифікаційних моделей (Logistic Regression, Gradient Boosting, Random Forest, Linear SVM, SVM), обрано **Gradient Boosting** та **Random Forest** як найкращі.
* **Крос-валідація:** Проведено порівняння кращих моделей на різних датасетах з використанням 5-кратної крос-валідації та метрики **F1-score** для вибору найбільш стабільного та продуктивного датасету.   

### **1.4. Валідація Викидів (Outlier Validation)**
Для перевірки впливу потенційних викидів на метрики найкращої моделі (Gradient Boosting або Random Forest на df1_mean) було проведено дослідження із використанням методу **Isolation Forest**.

* **Метод перевірки:** Послідовно визначалися потенційні викиди (від 0% до 10% зразків датасету) за допомогою **Isolation Forest**, виключалися з датасету, після чого обчислювалися основні метрики моделі.

* **Результати аналізу:** Поведінка метрик виявилася немонотонною та нагадувала статистичний шум. Ці коливання, ймовірно, пов'язані з формуванням різних (навчальних та тестових) вибірок при відкиданні різних частин датасету, а не з усуненням справжніх викидів.

* **Порівняльний аналіз:** Для підтвердження цієї гіпотези було проведено порівняння коливань метрик у двох випадках:

  1.   При послідовному виключенні різної кількості потенційних викидів.
  2.   При формуванні різного розподілу на навчальну та тестову частини базового датасету шляхом зміни `random state` в `train_test_split`.

* **Висновок:** В обох випадках метрики демонстрували **дуже схожу немонотонну поведінку** та діапазони значень. Різниця між основними статистичними характеристиками метрик (наприклад, середнє, стандартне відхилення) в обох випадках **не перевищувала 0,002**.

* **Загальний висновок:** Отримані результати свідчать про **відсутність суттєвих викидів** у робочому датасеті (`df1_mean`). Спостережувані коливання метрик обумовлені випадковими змінами у складі навчальних та тестових вибірок, а не впливом аномальних значень. Це підтверджує припущення про реальність спостережуваної неоднорідності даних (шум в даних) .

## 🛠️ **Аргументація вибору методів**
---

| Метод | Мета | Аргументація Вибору |
| :--- | :--- | :--- |
| **Видалення пропусків** | Обробка малих пропусків (0.5% у `download_avg`, `upload_avg`). | Дуже мала частка даних. Видалення не призведе до суттєвої втрати інформації, але забезпечить чистоту даних. |
| **Заповнення/Додаткова ознака** | Обробка великої кількості пропусків (30% у `reamining_contract`). | Збереження значного обсягу даних (30%). Введення бінарної ознаки (`reamining_contract_missing`) дозволяє моделі врахувати, що сам факт відсутності значення є важливим предиктором відтоку (кореляція 0.47). |
| **StandardScaler** | Нормалізація ознак. | Необхідна для моделей, чутливих до масштабу (наприклад, SVM, Logistic Regression, які також тестувалися) та для забезпечення рівного внеску всіх ознак у моделі. |
| **Gradient Boosting & Random Forest** | Базове моделювання. | Ці ансамблеві моделі на основі дерев рішень, як правило, добре працюють з "брудними" даними, нечутливі до викидів та нелінійностей, і є сильними базовими моделями для класифікації. |
| **Крос-валідація (F1-score)** | Оцінка надійності. | **F1-score** є більш релевантною метрикою, ніж проста точність (Accuracy), для задачі відтоку, де важливий баланс між Precision (точність прогнозування відтоку) та Recall (охоплення всіх клієнтів, що йдуть). Крос-валідація забезпечує стабільну оцінку продуктивності моделі на різних підмножинах даних. |

---

## 💡 **3. Ключові Знахідки**
---

* **Кореляція пропусків:** Факт відсутності значення в ознаці `reamining_contract` (відсутність контракту) має значну позитивну кореляцію з цільовою ознакою `churn` ($0.47$), що підтверджує важливість створення додаткової бінарної ознаки `reamining_contract_missing`.
* **Незначний вплив слабкокорельованих ознак:** Видалення ознак `bill_avg` та `service_failure_count` (низька лінійна кореляція) не призвело до покращення результатів, а в деяких випадках навіть погіршило їх, тому ці ознаки залишено в остаточному датасеті.
* **Перевага заповнення над видаленням:** Датасети, де пропуски в `reamining_contract` були заповнені (наприклад, `df1_mean`), показали трохи вищий та стабільніший **середній F1-score ($\approx 0.943 - 0.946$)** порівняно з датасетом, де ці записи були видалені (`df2`, $F1 \approx 0.932 - 0.936$).

## 💼 **4. Аналітичний Висновок Результатів (Що означає для бізнесу)**
---
| Аспект | Результат | Значення для Бізнесу |
| :--- | :--- | :--- |
| **Найкращий Датасет** | **`df1_mean`** (заповнення пропусків середнім + додаткова ознака про пропуск). | Для подальшої оптимізації моделі варто використовувати цей датасет, оскільки він дозволяє **зберегти найбільшу кількість даних** (на противагу `df2`), що потенційно забезпечує кращу генералізацію моделі. |
| **Базова Продуктивність** | Моделі **Gradient Boosting** та **Random Forest** досягли середнього **F1-score $\approx 94.6\%$** на найкращому датасеті. | Це **дуже високий** показник. Навіть на етапі базового моделювання компанія може ідентифікувати клієнтів, що знаходяться під загрозою відтоку, з високою точністю (94.6%). |
| **Фактор "Відсутність Контракту"** | Факт відсутності даних про залишок контракту (який ми виділили в ознаку) є сильним предиктором відтоку. | Компанія повинна звернути особливу увагу на клієнтів, для яких ця інформація відсутня. Це може свідчити про відсутність чинних абонентських угод і є критичним сигналом для відділу утримання. |

## 🚀 **Готовність до наступного етапу**

---

**Рекомендація:**

- Датасет **`df1_mean`** рекомендовано для подальшої **оптимізації гіперпараметрів** моделей **Gradient Boosting** та **Random Forest**.